In [3]:
import pandas as pd

# read the excel data into dataFrame
data = pd.read_excel("source_data.xlsx",index_col = 0, names=None )   # headers = None,sheet_name='setuptime',use_cols =

# df = pd.DataFrame(data, columns= ['M 1', 'M 2', 'M 3'])

# transform the dataframe to lists, each to use in the model
# print(data)
ptList = data.values.tolist()

# get the data from dataframe

# with row/column name
#print(df.at['Job 1','M 1'])

# with row/column index
# print(df.iat[1, 1])


# get the list data
# print(ptList)

         M 1    M 2    M 3    M 4    M 5
004UP     98    110    105  10000  10000
013QL     90    110    102  10000  10000
021AB     81     95     89  10000  10000
057FG  10000  10000  10000     61     59
059TH    100    120    112  10000  10000
078GH    150    175    165  10000  10000
078JK     91    105     99  10000  10000
089PW    114    133    125  10000  10000
089RT  10000  10000  10000     98     93
095EB  10000  10000  10000    112    108
[[98, 110, 105, 10000, 10000], [90, 110, 102, 10000, 10000], [81, 95, 89, 10000, 10000], [10000, 10000, 10000, 61, 59], [100, 120, 112, 10000, 10000], [150, 175, 165, 10000, 10000], [91, 105, 99, 10000, 10000], [114, 133, 125, 10000, 10000], [10000, 10000, 10000, 98, 93], [10000, 10000, 10000, 112, 108]]


In [4]:
# model, Gurobi
# unrelated parallel machine scheduling
from gurobipy import *


number_job = len(ptList)
number_machine = len(ptList[1])
processing_time = ptList



# Initial the gurobi model
m = Model()

#add variables 
x = m.addVars(number_machine,number_job,vtype=GRB.BINARY, name='x')
Cmax = m.addVar(lb = 0, name='makespan')

# Set the obj which means the minmize the makespan
m.setObjective(Cmax,GRB.MINIMIZE)

# add constraints
for i in range(number_machine):
    m.addConstr(quicksum(x[i,j]*processing_time[j][i] for j in range(number_job)) <= Cmax) 

for j in range(number_job):
    m.addConstr(quicksum(x[i,j] for i in range(number_machine)) == 1) 

m.optimize()


# Get the solution
# construct the 2-d array to store the jobs processed on the different machines
machine_list =[]
for i in range(number_machine):
    machine_list.append([])
for i in range(number_machine): 
    for j in range(number_job):
        if(x[i,j].X == 1):
            machine_list[i].append(j)
        
print(machine_list)

# get the start time and completion time for each job
m_time = [0] *number_machine
start =[0]  * number_job
finish =[0] * number_job
for i in range(len(machine_list)):
    for j in range(len(machine_list[i])):
        start[machine_list[i][j]] = m_time[i]
        finish[machine_list[i][j]] = m_time[i]+processing_time[machine_list[i][j]][i]
        m_time[i] = finish[machine_list[i][j]]
    
# Print the results （absolute time）
print (start)
print (finish)


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2021-05-11
Using license file /Users/liulei/gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 15 rows, 51 columns and 105 nonzeros
Model fingerprint: 0xf9f86182
Variable types: 1 continuous, 50 integer (50 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 20000.000000
Presolve time: 0.02s
Presolved: 15 rows, 51 columns, 105 nonzeros
Variable types: 0 continuous, 51 integer (50 binary)

Root relaxation: objective 2.584662e+02, 30 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    Be

In [ ]:
from datetime import datetime  
from datetime import timedelta

import plotly.express as px
import pandas as pd

# organize the data (absolute date time) to plotly
# suppose we use 2020-01-01 as the baseline time
start_date=[]
finish_date =[]

start_date_base = datetime.strptime('2020-01-01','%Y-%m-%d')

for i in range(number_job):
    start_date.append((start_date_base + timedelta(days=start[i])).strftime('%Y-%m-%d'))
    finish_date.append((start_date_base + timedelta(days=finish[i])).strftime('%Y-%m-%d'))

row_list = []
Job_index =0
for i in range(len(machine_list)):
    for j in range(len(machine_list[i])):
        dict1 = dict(Machine = "Machine"+str(i+1), Job  = "Job"+str(machine_list[i][j]+1), Start = start_date[machine_list[i][j]], Finish = finish_date[machine_list[i][j]])
        row_list.append(dict1)

df = pd.DataFrame(row_list)
print (df)

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Machine", color = 'Job')
fig.update_layout(showlegend=False) 
fig.show()